In [20]:
from pandas.io.parsers import read_csv
import numpy as np

np.random.seed(0) # REMOVE FOR FINAL SOLUTION


In [21]:
#Data Splitted.
def splitData(data):
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    testCount = int(shuffled.shape[0]*1/3)
    trainCount = shuffled.shape[0]-testCount
    trainSet = shuffled[0:trainCount]
    testSet = shuffled[trainCount:]

    return trainSet, testSet

def separateInputOutput(dataset):
    outputData = dataset[:,3] # Output
    inputData = np.concatenate((dataset[:,0:3],dataset[:,4:7],dataset[:,8:10]),axis=1) # Inputs
    return inputData, outputData

def getInputAttributeDict():
    return {   0 : 'calorific_value',
                    1 : 'nitrogen',
                    2 : 'turbidity',
                    3 : 'alcohol',
                    4 : 'sugars',
                    5 : 'bitterness',
                    6 : 'colour',
                    7 : 'degree_of_fermentation'} .copy()
    


#ML ALG
def initializeWeights(x,y):
    N = x.shape[0]
    
def normaliseData(dataArray, normType = "range"):
    normalisedDataList = []

    if normType == "range":
        maxVal = dataArray.max()
        minVal = dataArray.min()
        for val in dataArray:
            normalisedDataList.append((val-minVal)/(maxVal-minVal))
        return np.array(normalisedDataList)

    elif normType == "z":
        mean = dataArray.mean()
        stdDev =  getStandardDeviation(dataArray)
        for val in dataArray:
            normalisedDataList.append((val-mean)/stdDev)
        return np.array(normalisedDataList)

def getStandardDeviation(dataArray):
    n = dataArray.shape[0]
    mean = dataArray.mean()
    sumOfSquareDiffs = 0
    for val in dataArray:
        sumOfSquareDiffs += (val - mean)**2

    variance = sumOfSquareDiffs/(n-1)
    stdDev = variance**(1/2)
    return stdDev


In [22]:
#Reading in data
dataframe = read_csv("beer.txt",delim_whitespace=True,header=None)

# Preparing Data
dataset = dataframe.values
trainSet, testSet = splitData(dataset)
print(trainSet[0,7], trainSet[-1,7], testSet[0,7], testSet[-1,7])


testIn, testOut = separateInputOutput(testSet)
trainIn, trainOut = separateInputOutput(trainSet)

N =trainIn.shape[0]
Wi = 1/N
# print(getInputAttributeDict()[1])



91 22 47 20


In [23]:
def getUniqueClassCount(listToCount):
    classes = []
    for item in listToCount:
        if item not in classes:
            classes.append(item)

    counts = []
    for itemClass in classes:
        counts.append(listToCount.count(itemClass))
    return counts

def getLeafGini(counts):
    total = np.sum(counts)
    giniLeaf = 1
    for count in counts:
        giniLeaf -= (count/total)**2
    return giniLeaf

def getNodeGini(sortedList, threshold):
        lessThan = []
        greaterThan = []

        for i in range(len(sortedList)):
            x = sortedList[i][0]
            # print(x, threshold)
            if x < threshold:
                lessThan.append(sortedList[i][1])
            else:
                greaterThan.append(sortedList[i][1])

        countsLT = getUniqueClassCount(lessThan)
        countsGT = getUniqueClassCount(greaterThan)
        giniLT = getLeafGini(countsLT)
        giniGT = getLeafGini(countsGT)
        # print(giniLT,giniGT)
        # print(np.sum(countsLT),np.sum(countsGT))
        totalCount = len(sortedList)
        # print(totalCount)
        nodeGini = ((np.sum(countsLT)/totalCount)*giniLT) + ((np.sum(countsGT)/totalCount)*giniGT)
        # print(giniNode)
        return nodeGini


In [24]:
def getMinGiniAndThreshold(inputData, outputData, alreadyUsedAttributes):
    inputAttributesMinGinis = []
    attributeThresholds = []
    attributeIndexes = list(getInputAttributeDict())
    # print(attributeIndexes)
    for i in alreadyUsedAttributes:
        attributeIndexes.remove(i)
    print("Getting gini of attributes {}".format(attributeIndexes))
    
    for i in attributeIndexes:
        attribute = inputData[:, i].copy() # Single attribute from input data
        
        inputOutputPairs = []
        for j,sample in enumerate(attribute):
            inputOutputPairs.append((sample,trainOut[j]))
        attribute.sort()
        sortedInputOutputPairs = [tuple for x in attribute for tuple in inputOutputPairs if tuple[0] == x]

        testThresholds = []
        for j in range(len(attribute)-1):
            testThresholds.append((attribute[j]+attribute[j+1])/2)

        nodeGinis = []
        for testThresh in testThresholds:
            nodeGinis.append(getNodeGini(sortedInputOutputPairs, testThresh))

        minNodeGini = min(nodeGinis)
        minGiniIndex = nodeGinis.index(minNodeGini)
        attributeThreshold = testThresholds[minGiniIndex]

        inputAttributesMinGinis.append(minNodeGini)
        attributeThresholds.append(attributeThreshold)
    return inputAttributesMinGinis, attributeThresholds
   

In [25]:
def createTree(trainIn,trainOut):

    attributesUsed = []
    recursiveBranch(trainIn,trainOut, 1, attributesUsed)

    # minAttributeGinis, attributeThresholds = getMinGiniAndThreshold(trainIn,trainOut, [4,1])
    # print(minAttributeGinis, attributeThresholds)

    # rootAttributeGini = min(minAttributeGinis)
    # rootAttributeIndex = minAttributeGinis.index(rootAttributeGini)
    # print("Root is input data attribute", rootAttributeIndex)

    # rootThreshold = attributeThresholds[rootAttributeIndex]
    # unusedBranchAttributes = list(getInputAttributeDict())
    # print(unusedBranchAttributes)
    # unusedBranchAttributes.remove(rootAttributeIndex)
    # print(unusedBranchAttributes)

    # ltX,lty,gtX,gty = branchData(trainIn, trainOut, rootAttributeGini, rootThreshold, rootAttributeIndex, unusedBranchAttributes) #generate root
    return

def recursiveBranch(inputData, outputData, parentGini, attributesUsed):

    print(inputData.shape[0],"- Attributes Used =", attributesUsed)
    minAttributeGinis, attributeThresholds = getMinGiniAndThreshold(inputData, outputData, attributesUsed)
    nodeAttributeGini = min(minAttributeGinis)

    attributeIndexes = list(getInputAttributeDict())
    for i in attributesUsed:
        attributeIndexes.remove(i)
    # print("Indexes before removal of attribute just used",attributeIndexes)
    nodeAttributeIndex = attributeIndexes[minAttributeGinis.index(nodeAttributeGini)]
    attributesUsed.append(nodeAttributeIndex)

    
    nodeAttributeIndex = attributeIndexes[minAttributeGinis.index(nodeAttributeGini)]
    print("Node attribute index  (lowest gini) =",nodeAttributeIndex)

     # If none of the new ginis is less than parent gini, exit recursion
    if parentGini < nodeAttributeGini:
        print("Leaf created using attribute {} (has a min gini of {:.5f}).  Parent node is has gini = {:.5f}.".format(nodeAttributeIndex, nodeAttributeGini, parentGini))
        return
    else:            # Continue recursion
        nodeThreshold = attributeThresholds[minAttributeGinis.index(nodeAttributeGini)]
        print("Node created using attribute {} (has a min gini of {:.5f}). Threshold at this node = {}. Parent node has gini = {:.5f}.".format(nodeAttributeIndex, nodeAttributeGini, nodeThreshold, parentGini))

    ltX,lty,gtX,gty = branchData(inputData, outputData, nodeAttributeGini, nodeThreshold, nodeAttributeIndex, attributesUsed)
    return



In [26]:
def branchData(X, y, splitAttributeGini, splitValue, splitAttributeIndex, attributesUsed):
    lessThan_X = []
    lessThan_y = []
    greaterThan_X = []
    greaterThan_y = []
    # print("Split val and index", splitValue, splitAttributeIndex)
    for i in range(len(X)):
        if X[i,splitAttributeIndex] < splitValue:
            lessThan_X.append(X[i,:])
            lessThan_y.append(y[i])
        else:
            greaterThan_X.append(X[i,:])
            greaterThan_y.append(y[i])

    # Call recursively? :
        # if not at max depth:
            # Get min gini of new data for all attribs except this node's splitAttrib
            # if not less than splitAttributeGini, call recursive function
    
    # left branch first:
    # lessThan_X = np.delete(np.asarray(lessThan_X), splitAttributeIndex, 1)
    attributesUsed_left = attributesUsed.copy()
    attributesUsed_right = attributesUsed.copy()

    lessThan_X = np.asarray(lessThan_X)
    lessThan_y = np.asarray(lessThan_y)
    print("\nBranching left...")
    recursiveBranch(lessThan_X, lessThan_y, splitAttributeGini, attributesUsed_left)
    print("Finished left branch.\n")

    # Right branch
    # greaterThan_X = np.delete(np.asarray(greaterThan_X), splitAttributeIndex, 1)
    greaterThan_X = np.asarray(greaterThan_X)
    greaterThan_y = np.asarray(greaterThan_y)
    print("\nBranching right...")
    recursiveBranch(greaterThan_X, greaterThan_y, splitAttributeGini, attributesUsed_right)
    print("Finished right branch.\n")

    return lessThan_X,lessThan_y,greaterThan_X,greaterThan_y
        

In [27]:
createTree(trainIn,trainOut)

103 - Attributes Used = []
Getting gini of attributes [0, 1, 2, 3, 4, 5, 6, 7]
Node attribute index  (lowest gini) = 3
Node created using attribute 3 (has a min gini of 0.41523). Threshold at this node = 3.927692308. Parent node has gini = 1.00000.

Branching left...
39 - Attributes Used = [3]
Getting gini of attributes [0, 1, 2, 4, 5, 6, 7]
Node attribute index  (lowest gini) = 6
Leaf created using attribute 6 (has a min gini of 0.54358).  Parent node is has gini = 0.41523.
Finished left branch.


Branching right...
64 - Attributes Used = [3]
Getting gini of attributes [0, 1, 2, 4, 5, 6, 7]
Node attribute index  (lowest gini) = 0
Leaf created using attribute 0 (has a min gini of 0.61653).  Parent node is has gini = 0.41523.
Finished right branch.



In [28]:
class Node:
    def __init__(self,index,value):
        self.left = None
        self.right = None
        self.index = index
        self.value = value #stores value or the classification eg "ale"
        
    def addLeftChild(self,child):
        self.left = child
    
    def getLeftChild(self):
        return self.left

    def addRightChild(self,child):
        self.right = child

    def getRightChild(self):
        return self.right
    
    def printTree(self):
        if self.left:
            self.left.printTree()
        print(self.getData())
        if self.right:
            self.right.printTree()

    def getValue(self):
        return self.value
    
    def getData(self):
        return self.index,self.value
    
    def isLeaf(self):
        if (self.left == None) & (self.right == None):
            return True
        else:
            return False

In [29]:
#Traversing the tree with input data
def predict(X):
    predictedValues = []
    for i in range(len(X)):
        x = X[i]
        treePointer = Root
        index,value = treePointer.getData()
        while treePointer.isLeaf() == False:
            if (x[index] < value):
                treePointer = treePointer.getLeftChild()
                index,value = treePointer.getData()
            else:
                treePointer = treePointer.getRightChild()
                index,value = treePointer.getData()
        predictedValues.append(treePointer.getValue())
    return predictedValues

    


In [30]:
#Testing Tree
x = [12,11,13]
y = [[9,10,0],[13,11,0],[13,0,14]]
Root = Node(0,x[0])
currentNode =Root
Root.addLeftChild(Node(1,x[1]))
Root.addRightChild(Node(2,x[2]))
currentNode = Root.getLeftChild()
currentNode.addLeftChild(Node(None,"Ale"))
currentNode = Root.getRightChild()
currentNode.addLeftChild(Node(None,"larger"))
currentNode.addRightChild(Node(None,"Wine"))

# Root.printTree()
print(predict(y))


['Ale', 'larger', 'Wine']
